# RDD

an RDD is the fundamental data structure of Apache Spark. It's a fault-tolerant, distributed collection of elements that can be operated on in parallel.

**Key Characteristics:**

- Immutable
- Lazy evaluation
- Fault tolerant (via lineage info)
- Partitioned across cluster nodes
- Can be cached in memory

### SparkContext and SparkConf


SparkContext is the entry point for Spark functionality.

#### `SparkConf`

- Configuration for Spark application

**Common settings:**

- setMaster("local[*]") – Use local mode with all cores
- setAppName("RDDExample") – Application name

### transformations

Transformations create a new RDD from an existing one. They are lazy – not executed until an action is triggered.

| Transformation  | Description                                          |
| --------------- | ---------------------------------------------------- |
| `map(func)`     | Returns a new RDD by applying `func` to each element |
| `filter(func)`  | Filters elements for which `func` returns true       |
| `flatMap(func)` | Like map but flattens the result                     |
| `distinct()`    | Removes duplicates                                   |
| `union(rdd)`    | Combines two RDDs                                    |
| `groupByKey()`  | Groups values with same key                          |
| `reduceByKey()` | Aggregates values with same key using a function     |
| `sortBy(func)`  | Sorts RDD by computed key                            |


### actions

Actions trigger computation and return results or write data.

| Action             | Description                            |
| ------------------ | -------------------------------------- |
| `collect()`        | Returns all elements to driver         |
| `count()`          | Returns number of elements             |
| `first()`          | Returns first element                  |
| `take(n)`          | Returns first `n` elements             |
| `reduce(func)`     | Reduces elements using binary operator |
| `saveAsTextFile()` | Writes RDD to text files               |



reference - [spark rdd docs](https://spark.apache.org/docs/latest/rdd-programming-guide.html)


In [ ]:
! pip install pyspark

In [ ]:
# SparkContext and SparkConf

from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("salesDemo").setMaster("local[*]")

sc = SparkContext(conf=conf)




In [ ]:
sc.defaultParallelism

In [2]:
# Step 1: Load the file into RDD

sales_raw = sc.textFile("file:////workspace/TRNG-2224-data-engineering/week1/datasets/sales.txt")



In [3]:
# Step 2: Convert each line into a tuple (ProductID, Category, Amount)

records = sales_raw.map(lambda x: x.split(",")).map(lambda x: (int(x[0]), x[1], int(x[2])))




In [ ]:
# Step 3: Create a (Category, Amount) RDD

category_sales = records.map(lambda x: (x[1], x[2]) )


In [ ]:
# Step 4: Total sales by category

total_sales_by_category = category_sales.reduceByKey(lambda x,y: x+y)



In [ ]:
# Step 5: Average sale per category

average_sales_by_category = category_sales.mapValues(lambda x: (x, 1)).reduceByKey(lambda x,y: (x[0] + y[0], x[1]+ y[1])).mapValues(lambda x: x[0]/x[1])

average_sales_by_category.take(4)


In [ ]:
# Step 6: Highest transaction

max_tran = records.max(key=lambda x : x[2])

max_tran


In [ ]:
# highest selling category

highest_selling_category = total_sales_by_category.max(key=lambda x: x[1] )

highest_selling_category

In [ ]:
# Step 7: Categories with sales above 5000

high_selling_cat_5k = total_sales_by_category.filter(lambda x: x[1] > 5000)

high_selling_cat_5k.collect()




In [ ]:
# print final results

print("Total sales by cat")
print(total_sales_by_category.collect())

print("Avg sales per cat")

print(average_sales_by_category.collect())



**Assignment:**

1. find all product IDs where the amount is greater than 900.
2. Find all transactions that belong to the “Furniture” category.
3. Count how many transactions belong to the “Electronics” category.
4. Find average amount for each category.
5. Find the highest amount and the corresponding product ID.
6. Find the total number of unique categories.
7. For each category, find the product ID with the highest sale.
8. Count how many products were sold for less than 300.
9. Sort the transactions by amount in descending order.


### Shared Variables

When you pass a function (like in map or reduce) to Spark, that function runs on different machines in the cluster — not on your local driver.

**By default:**

- Spark makes separate copies of any variable you use inside those functions.
- So, if a task changes a variable on the executor, that change will not reflect back in your driver program.

This is done to keep things fast and distributed — but it also means you can not just update normal variables across tasks.

**challenges:**

You want to count how many rows have Amount > 500 using this code:

```py
count = 0

def increment_count(x):
    count += 1 

records.filter(lambda x: int(x[2]) > 500).foreach(increment_count)
print(count)

```

This will not work because each machine updates its own copy of count not the original one in the driver.

To solve such issues, spark provides two types of shared variables:

1. **Broadcast Variables:** a read-only variable that can be cached on each machine (executor). Used to efficiently share large data (like lookup tables) with all tasks without copying it multiple times.
2. **Accumulators:** variables used to safely implement counters or sums across mulitple worker nodes. You can only add to them (not read or subtract inside tasks). The final value is only accessible on the driver after an action is executed.




In [ ]:
# Load the dataset


In [ ]:
# Use a broadcast variable for region-level config


In [ ]:
# Use an accumulator to count high-value transactions



In [ ]:
# Save filtered RDD to a new file

### Assignment

**tasks:**

1.  Load the Dataset
2. Broadcast Variable: Category Discounts

```py
{"Electronics": 0.10, "Furniture": 0.15, "Clothing": 0.05, "Books": 0.20}
```

3. calculate and return the rdd:

```py
(ProductID, Region, Category, OriginalAmount, DiscountApplied)
```
4. Accumulator: Count Transactions Below ₹300
5. Filter and Save Results